In [1]:
import pandas as pd
import numpy as np
import os
import data_utility as du
import warnings
warnings.filterwarnings('ignore')
import requests

In [2]:
state_dict = {
    'nc': 'North Carolina',
    'pa': 'Pennsylvania',
    'ny': 'New York',
    'ne': 'Nebraska',
    'tx': 'Texas',
    'md': 'Maryland',
    'ky': 'Kentucky',
    'al': 'Alabama',
    'in': 'Indiana',
    'sc': 'South Carolina',
    'nd': 'North Dakota',
    'ar': 'Arkansas',
    'mn': 'Minnesota',
    'mo': 'Missouri',
    'mt': 'Montana',
    'tn': 'Tennessee',
    'ks': 'Kansas',
    'ga': 'Georgia',
    'me': 'Maine',
    'ia': 'Iowa',
    'il': 'Illinois',
    'ms': 'Mississippi',
    'fl': 'Florida',
    'co': 'Colorado',
    'nm': 'New Mexico',
    'va': 'Virginia',
    'ok': 'Oklahoma',
    'ak': 'Alaska',
    'mi': 'Michigan',
    'wv': 'West Virginia',
    'la': 'Louisiana',
    'az': 'Arizona',
    'oh': 'Ohio',
    'ri': 'Rhode Island',
    'ut': 'Utah',
    'de': 'Delaware',
    'ca': 'California',
    'wi': 'Wisconsin',
    'ma': 'Massachusetts',
    'sd': 'South Dakota',
    'id': 'Idaho',
    'or': 'Oregon',
    'wa': 'Washington',
    'dc': 'District of Columbia',
    'ct': 'Connecticut',
    'wy': 'Wyoming',
    'nj': 'New Jersey',
    'hi': 'Hawaii',
    'vt': 'Vermont',
    'nv': 'Nevada',
    'nh': 'New Hampshire'
}

dict_filename_url  = {
    "RDC_Inventory_Core_Metrics_State_History.csv" : "https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_State_History.csv",
    "RDC_Inventory_Core_Metrics_County_History.csv" : "https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_County_History.csv",
    "RDC_Inventory_Core_Metrics_Zip_History.csv"   : "https://econdata.s3-us-west-2.amazonaws.com/Reports/Core/RDC_Inventory_Core_Metrics_Zip_History.csv"
}

In [8]:
def get_files_from_web(url,filename):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        return "File downloaded successfully"
    else:
        return "Download Failed"


def state_data_pre_processing(mydata):
    mydata = mydata[:-1]
    mydata['date'] = pd.to_datetime( mydata['month_date_yyyymm'], format='%Y%m')

    col_name = ['date', 'state', 'median_listing_price', 'active_listing_count', 'median_listing_price_per_square_foot', 'median_square_feet',
     'average_listing_price', 'total_listing_count']


    mydata = mydata[col_name]

    mydata['median_listing_price'].fillna(mydata['median_listing_price'].median(),inplace=True)

    mydata['active_listing_count'].fillna(mydata['active_listing_count'].median(),inplace=True)

    mydata['median_listing_price_per_square_foot'].fillna(mydata['median_listing_price_per_square_foot'].median(),inplace=True)

    mydata['median_square_feet'].fillna(mydata['median_square_feet'].median(),inplace=True)

    mydata['average_listing_price'].fillna(mydata['average_listing_price'].mean(),inplace=True)

    mydata['total_listing_count'].fillna(mydata['average_listing_price'].median,inplace=True)

    mydata.insert(1,'month',mydata['date'].dt.month)

    mydata.insert(2,'year',mydata['date'].dt.year)

    mydata.sort_values(by=['state', 'year','month'],inplace=True,ignore_index=True)

    mydata = mydata.assign(
        state_median_listing_price_Previous_Month=mydata.groupby(['state', 'year'])['median_listing_price'].shift(+1),
        
        state_median_listing_price_Previous_Year_Month=mydata.groupby(['state','month'])['median_listing_price'].shift(+1),
        
        state_active_listing_count_Previous_Month=mydata.groupby(['state', 'year'])['active_listing_count'].shift(+1),
        
        state_active_listing_count_Previous_Year_Month=mydata.groupby(['state','month'])['active_listing_count'].shift(+1),
        
        state_median_listing_price_per_square_foot_Previous_Month=mydata.groupby(['state', 'year'])['median_listing_price_per_square_foot'].shift(+1),
        
        state_median_listing_price_per_square_foot_Previous_Year_Month=mydata.groupby(['state','month'])['median_listing_price_per_square_foot'].shift(+1),
        
        state_average_listing_price_Previous_Month=mydata.groupby(['state', 'year'])['average_listing_price'].shift(+1),
        
        state_average_listing_price_Previous_Year_Month=mydata.groupby(['state','month'])['average_listing_price'].shift(+1),
        
        state_total_listing_count_Previous_Month=mydata.groupby(['state', 'year'])['total_listing_count'].shift(+1),
        
        state_total_listing_count_Previous_Year_Month=mydata.groupby(['state','month'])['total_listing_count'].shift(+1)
    )

    mydata.fillna(0,inplace=True)
    
    return mydata


def county_data_pre_processing(mydata):
    mydata = mydata[:-1]
    
    mydata[['county','state']] = mydata['county_name'].str.split(',',expand=True)

    mydata['state'] = mydata['state'].apply(lambda x : x.strip())

    mydata['state'] = mydata['state'].map(state_dict)

    mydata['date'] = pd.to_datetime( mydata['month_date_yyyymm'], format='%Y%m')

    col_name = ['date', 'state', 'county', 'county_fips', 'median_listing_price', 'active_listing_count', 'median_listing_price_per_square_foot', 'median_square_feet',
     'average_listing_price', 'total_listing_count']


    mydata = mydata[col_name]

    mydata['median_listing_price'].fillna(mydata['median_listing_price'].median(),inplace=True)

    mydata['active_listing_count'].fillna(mydata['active_listing_count'].median(),inplace=True)

    mydata['median_listing_price_per_square_foot'].fillna(mydata['median_listing_price_per_square_foot'].median(),inplace=True)

    mydata['median_square_feet'].fillna(mydata['median_square_feet'].median(),inplace=True)

    mydata['average_listing_price'].fillna(mydata['average_listing_price'].mean(),inplace=True)

    mydata['total_listing_count'].fillna(mydata['average_listing_price'].median,inplace=True)

    mydata.insert(1,'month',mydata['date'].dt.month)

    mydata.insert(2,'year',mydata['date'].dt.year)

    mydata.sort_values(by=['county', 'state', 'year','month'],inplace=True,ignore_index=True)

    mydata = mydata.assign(
        median_listing_price_Previous_Month=mydata.groupby(['county', 'state', 'year'])['median_listing_price'].shift(+1),
        median_listing_price_Previous_Year_Month=mydata.groupby(['county', 'state', 'month'])['median_listing_price'].shift(+1),
        active_listing_count_Previous_Month=mydata.groupby(['county', 'state', 'year'])['active_listing_count'].shift(+1),
        active_listing_count_Previous_Year_Month=mydata.groupby(['county', 'state', 'month'])['active_listing_count'].shift(+1),
        median_listing_price_per_square_foot_Previous_Month=mydata.groupby(['county', 'state', 'year'])['median_listing_price_per_square_foot'].shift(+1),
        median_listing_price_per_square_foot_Previous_Year_Month=mydata.groupby(['county', 'state', 'month'])['median_listing_price_per_square_foot'].shift(+1),
        average_listing_price_Previous_Month=mydata.groupby(['county', 'state', 'year'])['average_listing_price'].shift(+1),
        average_listing_price_Previous_Year_Month=mydata.groupby(['county', 'state', 'month'])['average_listing_price'].shift(+1),
        total_listing_count_Previous_Month=mydata.groupby(['county', 'state', 'year'])['total_listing_count'].shift(+1),
        total_listing_count_Previous_Year_Month=mydata.groupby(['county', 'state', 'month'])['total_listing_count'].shift(+1)
    )

    mydata.fillna(0,inplace=True)
    
    for column in mydata.columns:
        mydata.rename(columns={column: f'county_{column}'}, inplace=True)
    
    return mydata



def zip_data_pre_processing(df,zipdata):
    
    df = df[:-1]
    
    result_list = list(set(df['postal_code'].unique()) - set(zipdata['Zip Code'].unique()))

    drop_index = list(df[(df['postal_code'].isin(result_list))].index)

    df.drop(index=drop_index,inplace=True)

    zipdata.rename(columns={"Zip Code": "postal_code"}, inplace=True)

    df['date'] = pd.to_datetime( df['month_date_yyyymm'], format='%Y%m')

    col_name = ['date', 'postal_code', 'zip_name', 'median_listing_price', 'active_listing_count', 'median_listing_price_per_square_foot', 'median_square_feet',
         'average_listing_price', 'total_listing_count']

    df = df[col_name]

    df['median_listing_price'].fillna(df['median_listing_price'].median(),inplace=True)

    df['active_listing_count'].fillna(df['active_listing_count'].median(),inplace=True)

    df['median_listing_price_per_square_foot'].fillna(df['median_listing_price_per_square_foot'].median(),inplace=True)

    df['median_square_feet'].fillna(df['median_square_feet'].median(),inplace=True)

    df['average_listing_price'].fillna(df['average_listing_price'].mean(),inplace=True)

    df['total_listing_count'].fillna(df['average_listing_price'].median,inplace=True)

    col_zip = ['postal_code','Official USPS city name','Primary Official County Name','Official USPS State Code','Official State Name']

    zipdata = zipdata[col_zip]

    df = df.merge(zipdata,on='postal_code',how='left')

    df.rename(columns={"Official USPS city name" : "city name",
                                  "Primary Official County Name" : "county name",
                                  "Official USPS State Code" : "state code",
                                  "Official State Name" : "state name"},inplace=True)

    df_col_name = ['date', 'postal_code', 'city name','county name', 'state code', 'state name', 'median_listing_price','active_listing_count', 'median_listing_price_per_square_foot','median_square_feet', 'average_listing_price', 'total_listing_count']

    df = df[df_col_name]
    
    df['county name'] = df['county name'].str.lower()
    
    df.insert(1,'month',df['date'].dt.month)

    df.insert(2,'year',df['date'].dt.year)

    df.sort_values(by=['postal_code', 'city name', 'county name', 'state name', 'year','month'],inplace=True,ignore_index=True)

    df = df.assign(
        median_listing_price_Previous_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name', 'year'])['median_listing_price'].shift(+1),
        
        median_listing_price_Previous_Year_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name','month'])['median_listing_price'].shift(+1),
        
        active_listing_count_Previous_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name', 'year'])['active_listing_count'].shift(+1),
        
        active_listing_count_Previous_Year_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name','month'])['active_listing_count'].shift(+1),
        
        medium_listing_price_per_square_foot_Previous_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name', 'year'])['median_listing_price_per_square_foot'].shift(+1),
        
        medium_listing_price_per_square_foot_Previous_Year_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name','month'])['median_listing_price_per_square_foot'].shift(+1),
        
        average_listing_price_Previous_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name', 'year'])['average_listing_price'].shift(+1),
        
        average_listing_price_Previous_Year_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name','month'])['average_listing_price'].shift(+1),
        
        total_listing_count_Previous_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name', 'year'])['total_listing_count'].shift(+1),
        
        total_listing_count_Previous_Year_Month=df.groupby(['postal_code', 'city name', 'county name', 'state name','month'])['total_listing_count'].shift(+1)
    )

    df.fillna(0,inplace=True)
    
    for column in df.columns:
        df.rename(columns={column : f'zip_{column}'}, inplace=True)
    
    return df

In [4]:
os.chdir('D:\\Datasets\\real_take\\Original_Files')
for x in dict_filename_url.keys():
    get_files_from_web(filename=x,url = dict_filename_url.get(x))

In [5]:
output_path = "D:\\Datasets\\real_take\\Files"
input_path = "D:\\Datasets\\real_take\\"

In [ ]:
state = du.get_data_from_files("RDC_Inventory_Core_Metrics_State_History.csv")
county =  du.get_data_from_files("RDC_Inventory_Core_Metrics_County_History.csv")
city = du.get_data_from_files('RDC_Inventory_Core_Metrics_Zip_History.csv'),du.get_data_from_files(input_path + '\\georef-united-states-of-america-zc-point.csv',sep=';')

In [9]:
state_data_pre_processing(state).to_csv(output_path + "\\State_History_data.csv",index=False)

county_data_pre_processing(county).to_csv(output_path + "\\County_History_data.csv",index=False)

zip_data_pre_processing(city).to_csv(output_path + "\\zipcode_History_data.csv",index=False)